In [29]:
import pandas as pd
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import matplotlib.pyplot as plt
import plotly.graph_objects as go

In [30]:
#añadimos el vader lexicon para interpretar frases en desorden
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\sachs\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [34]:
df_details= pd.read_csv('Places_Details.csv')
df_reviews = pd.read_csv('places_reviews.csv')

#Funcion para contar raitings y opiniones positivas y negativas de restaurantes.
def count_keywords_ratings_details_and_sentiment(df_reviews, df_details):
    #se implementa un metodo para tratar de identificar los sentimientos en las reviews
    sia = SentimentIntensityAnalyzer()
    #Compilacion simple de palabras/frases cortas comunes que nos pueden inicar como se sienten lxs comensales acerca del 
    #restaurante en 7 categorias distintas. 
    keywords = {
        'sabor': {
            'positive': ['delicioso', 'sabor auténtico', 'sabores increíbles'],
            'negative': ['insipido', 'salado', 'desagradable']
        },
        'variedad': {
            'positive': ['muchas opciones', 'amplia selección', 'variedad impresionante'],
            'negative': ['pocas opciones', 'menú limitado', 'falta de variedad']
        },
        'servicio': {
            'positive': ['servicio excelente', 'personal amable', 'rápido','eficiente'],
            'negative': ['lento', 'grosero', 'mal servicio']
        },
        'ambiente': {
            'positive': ['ambiente agradable', 'buena decoración', 'acogedor'],
            'negative': ['ruidoso', 'mala decoración', 'incómodo']
        },
        'precio': {
            'positive': ['razonable', ' calidad-precio', 'asequible'],
            'negative': ['caro', 'precios exorbitantes', 'no vale la pena']
        },
        'higiene': {
            'positive': ['limpio', 'bien mantenido', 'impecable'],
            'negative': ['sucio', 'mal mantenido', 'desordenado']
        },
        'ubicación': {
            'positive': ['ubicación conveniente', 'fácil de encontrar', 'bien situado'],
            'negative': ['ubicación incómoda', 'difícil de encontrar', 'mal situado']
        }
    }
    ## Ratings de cuantas estrellas se le pueden dar a cada restaurante
    ratings = [1, 2, 3, 4, 5]
    ##Buscamos todas id únicas de restaurantes  
    place_ids = df_reviews['place_id'].unique()

    #Generacion de un diccionario con los raitings, detalles y sentimeintos, con todos los restaurantes
    keyword_rating_detail_and_sentiment_counts = {id: {keyword: {'positive': 0, 'negative': 0} for keyword in keywords} for id in place_ids}

    #ponemos todo en 0 uwu
    for id in place_ids:
        keyword_rating_detail_and_sentiment_counts[id].update({rating: 0 for rating in ratings})
        
    #extremos la informacion de nuesto DF, id del restaurante, el texto de la review y las estrellas 
    for index, row in df_reviews.iterrows():
        place_id = row['place_id']
        review = row['text'].lower()
        rating = row['rating']
        #definimos la polarity score del modulo de sia para intentar dicernir si el sentimiento a la categoria asociada es positivo o negativo
        polarity_score = sia.polarity_scores(review)
        sentiment = 'positive' if polarity_score['compound'] > 0 else 'negative'
        for keyword, sentiments in keywords.items():
            for sentiment, phrases in sentiments.items():
                for phrase in phrases:
                    if phrase in review:
                        #despues de evaluar el sentimiento y precencia de palabras clave damos un punto a la categoria asociada
                        keyword_rating_detail_and_sentiment_counts[place_id][keyword][sentiment] += 1
        if rating in ratings:
            #recorremos cuantas estrellas da la review y damos un punto segun la cantidad de estrellas
            keyword_rating_detail_and_sentiment_counts[place_id][rating] += 1
            
    ## se cree ordena el diccionario
    for index, row in df_details.iterrows():
        place_id = row['place_id']
        name = row['name']
        average_rating = row['rating']
        total_user_ratings = row['user_ratings_total']
        if place_id in keyword_rating_detail_and_sentiment_counts:
            keyword_rating_detail_and_sentiment_counts[place_id].update({'name': name, 'average_rating': average_rating, 'total_user_ratings': total_user_ratings})
            keyword_rating_detail_and_sentiment_counts[name] = keyword_rating_detail_and_sentiment_counts[place_id]

    # convertimos el diccionario en un DF y hacemos un CSV para tener los datos a la mano
    df_output = pd.DataFrame.from_dict(keyword_rating_detail_and_sentiment_counts, orient='index')
    df_output.to_csv('Opiniones.csv')

    #Return del diccionario con el que se estaba trabajando
    return keyword_rating_detail_and_sentiment_counts

keyword_rating_detail_and_sentiment_counts = count_keywords_ratings_details_and_sentiment(df_reviews, df_details)

#Busquedas de ejemplo
print(keyword_rating_detail_and_sentiment_counts['TORTAS AHOGADAS EL CHIFLADERO GDL'])
print(keyword_rating_detail_and_sentiment_counts['ChIJ4ftOXEH-zYUREp_SkJ49kVI'])
print(keyword_rating_detail_and_sentiment_counts['Tlayudas el Tasajo'])
print(keyword_rating_detail_and_sentiment_counts['ChIJa1lZNZ__0YURjDHz4gNWVQc'])


{'sabor': {'positive': 2, 'negative': 2}, 'variedad': {'positive': 0, 'negative': 0}, 'servicio': {'positive': 0, 'negative': 0}, 'ambiente': {'positive': 0, 'negative': 0}, 'precio': {'positive': 0, 'negative': 0}, 'higiene': {'positive': 1, 'negative': 0}, 'ubicación': {'positive': 0, 'negative': 0}, 1: 1, 2: 1, 3: 2, 4: 12, 5: 70, 'name': 'TORTAS AHOGADAS EL CHIFLADERO GDL', 'average_rating': 4.6, 'total_user_ratings': 389}
{'sabor': {'positive': 2, 'negative': 2}, 'variedad': {'positive': 0, 'negative': 0}, 'servicio': {'positive': 0, 'negative': 0}, 'ambiente': {'positive': 0, 'negative': 0}, 'precio': {'positive': 0, 'negative': 0}, 'higiene': {'positive': 1, 'negative': 0}, 'ubicación': {'positive': 0, 'negative': 0}, 1: 1, 2: 1, 3: 2, 4: 12, 5: 70, 'name': 'TORTAS AHOGADAS EL CHIFLADERO GDL', 'average_rating': 4.6, 'total_user_ratings': 389}
{'sabor': {'positive': 8, 'negative': 1}, 'variedad': {'positive': 0, 'negative': 0}, 'servicio': {'positive': 1, 'negative': 0}, 'ambient

In [32]:
#Funcion para plotear opiniones positivas y negativas en las 7 metricas definidas
def plot_reviews(place_id, counts):
    # Tomamos la id del lugar del output de la funcion previa
    place_counts = counts[place_id]

    # Separamos las cuentas de opiniones positivas y negativas
    positive_counts = {keyword: count['positive'] for keyword, count in place_counts.items() if isinstance(count, dict)}
    negative_counts = {keyword: count['negative'] for keyword, count in place_counts.items() if isinstance(count, dict)}

    # Histograma :)
    fig = go.Figure()
    fig.add_trace(go.Bar(x=list(positive_counts.keys()), y=list(positive_counts.values()), name='Positive', marker_color='#1FB015'))
    fig.add_trace(go.Bar(x=list(negative_counts.keys()), y=list(negative_counts.values()), name='Negative', marker_color='#E73048'))
    fig.update_layout(barmode='group', xaxis_tickangle=-45, title_text='Positive and Negative Reviews for Each Category')
    fig.show()

## Ejemplo
plot_reviews('ChIJa1lZNZ__0YURjDHz4gNWVQc', keyword_rating_detail_and_sentiment_counts)

In [33]:
#Funcion para plotear cuantas reviews de 1 a 5 estrellas tiene cada restaurante
def plot_star_ratings(place_id, counts):
     # Tomamos la id del lugar del output de la Primer funcion
    place_counts = counts[place_id]

    # Separamos las columans por estrellas
    star_rating_counts = {rating: count for rating, count in place_counts.items() if isinstance(rating, int)}

    # Histograma uwu
    fig = go.Figure()
    fig.add_trace(go.Bar(x=list(star_rating_counts.keys()), y=list(star_rating_counts.values()), name='Star Ratings', marker_color='#F8BE13'))
    fig.update_layout(title_text='Number of Reviews with Each Star Rating')
    fig.show()
    
##Ejemplo (Mismo Restaurante)
plot_star_ratings('ChIJa1lZNZ__0YURjDHz4gNWVQc', keyword_rating_detail_and_sentiment_counts)